In [179]:
using DataFrames
using CSV
using XLSX
using Statistics
using Plots
using Dates
using GLM
using MLBase
using LinearAlgebra

In [2]:
prices = XLSX.readtable("data/zillow_data_download_april2020.xlsx", "Sales_median_price_city", "A:ER")
counts = XLSX.readtable("data/zillow_data_download_april2020.xlsx", "Sale_counts_city", "A:ER")

#Columns we care about
atr = 1:4 #regionID, city, state, size atributes
years = 39:147 #2011-2020
colIndex = [atr; years]

pricesDF = DataFrame(prices[1][colIndex], prices[2][colIndex])
pricesData = dropmissing(pricesDF)

countsDF = DataFrame(counts[1][colIndex], counts[2][colIndex])
countsData = dropmissing(countsDF) #So we don't have to re-read to get old data

,RegionID,RegionName,StateName,SizeRank,2011-01,2011-02,2011-03
,Any,Any,Any,Any,Any,Any,Any
1,6181,New York,New York,1,2821,2865,3303
2,12447,Los Angeles,California,2,1834,1723,2391
3,39051,Houston,Texas,3,1960,1762,2707
4,17426,Chicago,Illinois,4,2111,1792,2457
5,6915,San Antonio,Texas,5,962,960,1372
6,13271,Philadelphia,Pennsylvania,6,961,981,1334
7,40326,Phoenix,Arizona,7,2529,2585,3558
8,18959,Las Vegas,Nevada,8,2358,2574,3127
9,54296,San Diego,California,9,876,924,1157


In [3]:
function evalSin(X, A)
    a = A[1]
    b = A[2]
    c = A[3]
    d = A[4]
    
    return a.*sin.(b.*X.+c).+d
end

function evalJacob(X, A)
    a = A[1]
    b = A[2]
    c = A[3]
    d = A[4]
    J = []
    
    for x in X
        da = sin(b*x + c)
        db = a*x*cos(b*x+c)
        dc = a*cos(b*x+c)
        
        if isempty(J)
            J = [da db dc 1]
        else
            J = [J; da db dc 1]
        end
    end
    
    return J
end

function newton(data, a0)
    alp = 0.05
    tol = 1e-2
    count = 0
    
    a0 = a0'
    R = data.Y .- evalSin(data.X, a0)
    J = evalJacob(data.X, a0)
    
    pInv = inv(J'*J)*J'
    A = a0 .+ alp*pInv*R

    while norm(a0-A, 2) > tol
        a0 = A
        
        R = data.Y .- evalSin(data.X, a0)
        J = evalJacob(data.X, a0)
        pInv = inv(J'*J)*J'
        
        A = a0 .+ alp*pInv*R
        
        count = count + 1
        
        if count > 5e4
            print("Broke with count of >1e5")
            break
        end
    end
    
    return A
end

newton (generic function with 1 method)

In [445]:
#Make initial plots
analysisDF = DataFrame(
    cityName = String[], stateName = String[], 
    slope = Float64[], inter = Float64[], homePriceMean = Float64[], 
    amplitude = Float64[], shift = Float64[], fitError = Float64[])

#Vectorize input dates
datesInput = collect(Date(2011, 1):Dates.Month(1):Date(2020, 1))

#Top N cities by population we are going to analyze
N = 700

for row = 1:N
    #Vectorize inputs and outputs
    time = 1:size(datesInput)[1]
    countsOutput = collect(countsData[row, 5:end])
    pricesOutput = collect(pricesData[row, 5:end])
    
    #Get city and state name
    cityName = countsData[row, 2]
    stateName = countsData[row, 3]  
    loc = string(cityName, ", ", stateName)
    
    #Two regressions: Linear and Sinusodial
    
    #Linear Regression
    priceFitDF = DataFrame(t = time, medPrice = pricesOutput)
    priceFit = lm(@formula(medPrice ~ t), priceFitDF)
   
    homePriceMean = mean(pricesOutput)
    slope = round(GLM.coef(priceFit)[2], digits = 3)
    inter = round(GLM.coef(priceFit)[1], digits = 3)
    linFitOutput = predict(priceFit)   
    
    #Sin Regressoin
    soldMax = maximum(countsOutput)
    soldMean = mean(countsOutput)
    soldN = size(countsData)[1]
    a0 = [soldMax/2 0.5 0 soldMax/2] #Initial guess (period of 2pi/12 ~= 0.5)    

    A = newton(DataFrame(X = time, Y = countsOutput), a0)    
    fit = evalSin(time, A)
    
    #Root relative error 
    a = sum((fit .- countsOutput).^2)
    b = sum((soldMean .- countsOutput).^2)
    fitError = sqrt(a/b)
    
    #Add new row to dataframe
    push!(analysisDF, 
            [cityName, stateName, 
             slope, inter, homePriceMean, 
             A[1], A[3], fitError])
    
    #Plot: units sold and best fit, median price and best fit
    plot(datesInput, countsOutput, 
        title = loc, xlabel = "Month", ylabel = "Homes Sold", label = "Zillow Data", legend = :topleft)
    plot!(datesInput, evalSin(time, A), linecolor = :black, linestyle = :dash, label = "Newton-Gauss")
    png(string("Plots/Homes-Sold/", string(loc, "-Homes-Sold")))
    
        
    scatter(datesInput, pricesOutput, 
        title = loc, xlabel = "Month", ylabel = "Median Price (\$)", label = "Zillow Data")
    plot!(datesInput, linFitOutput, lw = 3, label = "Lin. Reg", legend = :bottomright, linestyle = :dash)
    annotate!(datesInput[1], maximum(pricesOutput), Plots.text(string("m = ", slope), 10, :black, :left))
    png(string("Plots/Median-Price/", string(loc, "-Median-Price")))
end

In [446]:
maxInter = maximum(analysisDF.inter)

range = [0, 2e5, 3e5, 4e5, 5e5, 6e5, 1e6]
nbins = size(range)[1]
dfRows = size(analysisDF)[1]

#Create marketRank column
analysisDF.marketRank = zeros(dfRows)
analysisDF.rankedGrowth = zeros(dfRows)

#Split the cities in bins by mean median home price
for i = 1:nbins
    if i == 1
        analysisDF[analysisDF.inter .< range[i+1], :marketRank] = i
    elseif i == nbins
        analysisDF[analysisDF.inter .> range[i], :marketRank] = i
    else
        analysisDF[range[i] .< analysisDF.inter .< range[i+1], :marketRank] = i
    end
end

#Rank the growth rate for each ranked market
#We take the mean price growth rate for all cities within a price range
#Then use it to standardize growth rates for cities within that rank
for i = 1:dfRows
    rank = analysisDF[i, :marketRank]
    for j = 1:nbins
        if rank == j
            indicies = analysisDF.marketRank .== rank
            analysisDF[i, :rankedGrowth] = analysisDF[i, :slope]/mean(analysisDF[indicies, :slope])
        end
    end
end

topMarkets = DataFrame()

#Create plot of most interesting markets (i.e. low volatility and high relative growth rate)
for i = 1:dfRows
    if analysisDF[i, :fitError] <= 0.7 && analysisDF[i, :rankedGrowth] >= 0
        if isempty(topMarkets)
            topMarkets = DataFrame(analysisDF[i, :])
        else
            push!(topMarkets, (analysisDF[i, :]))
        end
    end
end

CSV.write("Analysis.csv", analysisDF)
CSV.write("TopMarkets.csv", topMarkets)

scatter(topMarkets.fitError, topMarkets.rankedGrowth, ylabel = "Ranked Growth", xlabel = "Market Volatility",
    series_annotations = Plots.text.(topMarkets.cityName, 7, :left), legend = false)
png("TopMarkets.png")

┌ Warning: implicit broadcasting in setindex! is deprecated; use `df[row_inds, col_ind] .= Ref(v)` broadcasting assignment to change the column in place
│   caller = top-level scope at In[446]:14
└ @ Core ./In[446]:14
┌ Warning: implicit broadcasting in setindex! is deprecated; use `df[row_inds, col_ind] .= Ref(v)` broadcasting assignment to change the column in place
│   caller = top-level scope at In[446]:18
└ @ Core ./In[446]:18
┌ Warning: implicit broadcasting in setindex! is deprecated; use `df[row_inds, col_ind] .= Ref(v)` broadcasting assignment to change the column in place
│   caller = top-level scope at In[446]:16
└ @ Core ./In[446]:16
